In [ ]:
#%%import packages
import numpy as np
import pandas as pd
from scipy import linalg
from scipy.stats import gamma
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt

In [ ]:
#%% get data
df=pd.read_csv('greenbuildings.csv')
y = ((df['Rent']*df['leasing_rate'])/100).to_frame().to_numpy()
X = df[['green_rating', 'City_Market_Rent', 'age','class_a', 'class_b']].to_numpy()

In [ ]:
#%% For graphs
import matplotlib.style as style
plt.rcParams.update({'axes.labelsize':16})
plt.rcParams.update({'axes.titlesize':16})
plt.rcParams.update({'legend.fontsize':16})
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['lines.linewidth'] = 4
style.use('ggplot')

In [ ]:
from linear_reg import LinearModel_Bayes

In [ ]:
model1 = LinearModel_Bayes(X, y)
model1.fit_homoskedastic(fit_intercept=True)

In [ ]:
#get built-in package estimate to compare
import statsmodels.api as sm
X_add = sm.add_constant(X)
mod = sm.OLS(y, X_add)
fit = mod.fit()
beta_package = fit.params
intervals_package = fit.conf_int(alpha=0.05, cols=None)

In [ ]:
#1) get mean of posterior estimate for the parameters
beta_estimate = model1.m_star
#plot 
fig, ax = plt.subplots()
ax.plot([i+1 for i in range(len(beta_estimate))], beta_estimate, 'bo', linestyle='dashed', linewidth=2.0, label='My model')
ax.plot([i+1 for i in range(len(beta_estimate))], beta_package, linewidth=2.0, label='Package')
plt.title("Homoskedastic model - estimates")
plt.xlabel("coefficient index")
plt.ylabel("estimate")
plt.legend()
plt.show()

In [ ]:
#2) get confidence intervls simulating multivariate t
def multivariatet(mu,Sigma,N,M):
    '''
    Output:
    Produce M samples of d-dimensional multivariate t distribution
    Input:
    mu = mean (d dimensional numpy array or scalar)
    Sigma = scale matrix (dxd numpy array)
    N = degrees of freedom
    M = # of samples to produce
    '''
    d = len(Sigma)
    g = np.tile(np.random.gamma(N/2.,2./N,M),(d,1)).T
    Z = np.random.multivariate_normal(np.zeros(d),Sigma,M)
    return mu + Z/np.sqrt(g)
mean = model1.m_star.flatten()
scale = model1.scale_star
df = 7820
sample = multivariatet(mean, scale, df, 10000)
sorted_matrix = np.sort(sample, axis=0)
conf_interval = sorted_matrix[250,:], sorted_matrix[850,:]


In [ ]:
# plot
width = [i+1 for i in range(len(conf_interval[0]))]
fig, ax = plt.subplots()
ax.plot(width, conf_interval[0], 'bo', linestyle='dashed', linewidth=2.0,  label = 'My model',  color='red',)
ax.plot(width, conf_interval[1], 'bo', linestyle='dashed', linewidth=2.0,   color='red')
ax.plot(width, intervals_package[:,0], linewidth=2.0, color ='blue', label='package')
ax.plot(width, intervals_package[:,1], ls='-', linewidth=2.0, color='blue')
plt.title("Homoskedastic model - intervals")
plt.xlabel("coefficient index")
plt.ylabel("confidence intervals")
plt.legend()
plt.show()

In [ ]:
#3) histogram of residuals
X_int = np.hstack((np.ones(X.shape[0])[:, None], X))#add intercept to X
residuals = y- X_int @ beta_estimate
fig, ax = plt.subplots()
ax.hist(residuals, bins=200, color='purple')
plt.title("Homoskedastic model - histogram of residuals")
plt.show()

In [ ]:
#comment
#you can see that for some of them you have poor explanation. you would like to not have those high errors
#could be non normality of the model (hence not normality of the errors)
#or could be heteroskedasticity
#how can I test for 
#%% look at residuald agains city market rent. we see that when citymakert low you get small values close to 0. good, but growing when citymarket grows
#you see that there is heteroskedasticiy, but one thing doesn't exclude the other 